In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rasterio rioxarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 117.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 4.2 MB/s eta 0:00:00


In [3]:
import os
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
from glob import glob

In [4]:


# 📁 Ruta a la carpeta de escenas
data_folder = "/content/drive/MyDrive/Programming/Colab Notebooks/Fray Jorge LULC/FrayJorge/input/S2/Sentinel2_Exports"

# 🔍 Encuentra todas las fechas
files = glob(os.path.join(data_folder, "*.tif"))
dates = sorted(list(set([os.path.basename(f).split("_")[2] for f in files])))

# 📤 Carpeta de salida
output_folder = os.path.join(data_folder, "processed")
os.makedirs(output_folder, exist_ok=True)

def merge_tiles(file_pattern):
    """Merge mosaics G + F for a single band type."""
    matching_files = sorted(glob(file_pattern))
    srcs = [rasterio.open(fp) for fp in matching_files]
    merged, out_trans = merge(srcs)
    out_meta = srcs[0].meta.copy()
    out_meta.update({
        "height": merged.shape[1],
        "width": merged.shape[2],
        "transform": out_trans
    })
    for src in srcs:
        src.close()
    return merged, out_meta

def resample_to_10m(src_array, src_meta, target_shape, target_transform, method=Resampling.bilinear):
    """Resample input array to match 10m target resolution."""
    dst_array = np.empty(shape=target_shape, dtype=src_meta['dtype'])
    reproject(
        source=src_array,
        destination=dst_array,
        src_transform=src_meta['transform'],
        src_crs=src_meta['crs'],
        dst_transform=target_transform,
        dst_crs=src_meta['crs'],
        resampling=method
    )
    return dst_array

# 🧠 Proceso por cada fecha
for date in dates:
    print(f"Procesando fecha: {date}")

    # Merge por tipo y mosaico (G/F)
    merged_data = {}
    for res in ["10m", "20m", "60m", "SCL"]:
        pattern = os.path.join(data_folder, f"S2_{res}_{date}_*.tif")
        arr, meta = merge_tiles(pattern)
        merged_data[res] = {"array": arr, "meta": meta}

    # Usamos como referencia la resolución 10m
    base_meta = merged_data["10m"]["meta"]
    base_shape = merged_data["10m"]["array"].shape
    base_transform = base_meta["transform"]

    # Resamplea bandas 20m y 60m a 10m
    bands_resampled = [merged_data["10m"]["array"]]  # ya está a 10m
    for res in ["20m", "60m"]:
        arr = merged_data[res]["array"]
        meta = merged_data[res]["meta"]
        for band in arr:
            band_resampled = resample_to_10m(
                band,
                meta,
                target_shape=base_shape[1:],  # (height, width)
                target_transform=base_transform
            )
            bands_resampled.append(band_resampled[np.newaxis, :, :])  # agregar como banda nueva

    # Agrega banda SCL sin reescalar
        # === SCL ===
    scl_array = merged_data["SCL"]["array"]
    scl_meta = merged_data["SCL"]["meta"]
    scl_resampled = resample_to_10m(
        scl_array[0],  # Solo una banda
        scl_meta,
        target_shape=base_shape[1:],
        target_transform=base_transform,
        method=Resampling.nearest  # no interpolar clasificaciones
    )
    bands_resampled.append(scl_resampled[np.newaxis, :, :])  # una banda extra

    # Stack final
    full_stack = np.concatenate(bands_resampled, axis=0)
    final_meta = base_meta.copy()
    final_meta.update({
        "count": full_stack.shape[0],
        "dtype": full_stack.dtype,
        "driver": "GTiff"
    })

    # Nombres para las bandas (ejemplo: B1_10m, B1_20m...)
    #band_names = (
    #    [f"B{i+1}_10m" for i in range(merged_data["10m"]["array"].shape[0])] +
    #    [f"B{i+1}_20m" for i in range(merged_data["20m"]["array"].shape[0])] +
    #    [f"B{i+1}_60m" for i in range(merged_data["60m"]["array"].shape[0])] +
    #    ["SCL"]
    #)
    band_names = (
     ['B2', 'B3', 'B4', 'B8'] +
     ['B5', 'B6', 'B7', 'B8A', 'B11', 'B12'] +
     ['B1', 'B9'] +
     ['SCL']
    )
    # Guardar archivo
    out_path = os.path.join(output_folder, f"S2_stack_{date}_10m.tif")
    with rasterio.open(out_path, "w", **final_meta) as dst:
        for i in range(full_stack.shape[0]):
            dst.write(full_stack[i, :, :], i + 1)
            dst.set_band_description(i + 1, band_names[i])

    print(f"✔️ Raster guardado: {out_path}")


Procesando fecha: 2024-03-08
✔️ Raster guardado: /content/drive/MyDrive/Programming/Colab Notebooks/Fray Jorge LULC/FrayJorge/input/S2/Sentinel2_Exports/processed/S2_stack_2024-03-08_10m.tif
Procesando fecha: 2024-03-21
✔️ Raster guardado: /content/drive/MyDrive/Programming/Colab Notebooks/Fray Jorge LULC/FrayJorge/input/S2/Sentinel2_Exports/processed/S2_stack_2024-03-21_10m.tif
Procesando fecha: 2024-04-02
✔️ Raster guardado: /content/drive/MyDrive/Programming/Colab Notebooks/Fray Jorge LULC/FrayJorge/input/S2/Sentinel2_Exports/processed/S2_stack_2024-04-02_10m.tif
Procesando fecha: 2024-04-17
✔️ Raster guardado: /content/drive/MyDrive/Programming/Colab Notebooks/Fray Jorge LULC/FrayJorge/input/S2/Sentinel2_Exports/processed/S2_stack_2024-04-17_10m.tif
Procesando fecha: 2024-04-22
✔️ Raster guardado: /content/drive/MyDrive/Programming/Colab Notebooks/Fray Jorge LULC/FrayJorge/input/S2/Sentinel2_Exports/processed/S2_stack_2024-04-22_10m.tif
Procesando fecha: 2024-04-25
✔️ Raster guarda

Read each scene and stack to save into a single dataset.

In [4]:
data_folder = "/content/drive/MyDrive/Programming/Colab Notebooks/Fray Jorge LULC/FrayJorge/input/S2/Sentinel2_Exports"

# 🔍 Encuentra todas las fechas

# 📤 Carpeta de salida
output_folder = os.path.join(data_folder, "processed")


In [5]:
import os
import xarray as xr
import rioxarray as rxr
import pandas as pd
from glob import glob

# 1. Folder with rasters
data_folder = output_folder  # UPDATE THIS
raster_files = sorted(glob(os.path.join(data_folder, "S2_stack_*.tif")))

# 2. Read each raster as an xarray.DataArray, assign time from filename
data_arrays = []
times = []

for rf in raster_files:
    # Extract date from filename
    basename = os.path.basename(rf)
    date_str = basename.split("_")[2]
    time = pd.to_datetime(date_str)

    # Read raster with rioxarray
    da = rxr.open_rasterio(rf, masked=True)  # shape: (bands, y, x)

    if "band" in da.dims:
        band_coord_values = da.coords.get("band").values.tolist()
        # Check if there are any band coordinates and if the first one is not a string starting with 'B'
        if band_coord_values and (not isinstance(band_coord_values[0], str) or not band_coord_values[0].startswith("B")):
             # You can define your band names manually or dynamically
            band_names = ['B2', 'B3', 'B4', 'B8', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12', 'B1', 'B9', 'SCL']
            # Ensure the number of band names matches the number of bands in the DataArray
            if len(band_names) >= da.shape[0]:
                 da = da.assign_coords(band=band_names[:da.shape[0]])
            else:
                 print(f"Warning: Not enough predefined band names for raster {rf}. Using default band numbers.")

    # Add a new time dimension
    da = da.expand_dims(time=[time])  # shape: (time, band, y, x)
    data_arrays.append(da)
    times.append(time)

# 3. Concatenate all along the time axis
ds = xr.concat(data_arrays, dim="time")

# Optional: rename 'band' dimension to 'variable' for clarity
ds = ds.rename({'band': 'variable'})

# Final dataset has dimensions: time x variable x y x x
print(ds)


<xarray.DataArray (time: 14, variable: 13, y: 8020, x: 2335)> Size: 14GB
array([[[[2.380e+02, 2.330e+02, 2.490e+02, ..., 8.310e+02, 8.180e+02,
          8.550e+02],
         [2.540e+02, 2.360e+02, 2.200e+02, ..., 8.140e+02, 8.030e+02,
          8.330e+02],
         [2.540e+02, 2.300e+02, 2.220e+02, ..., 8.310e+02, 8.440e+02,
          8.070e+02],
         ...,
         [2.360e+02, 2.190e+02, 2.160e+02, ..., 6.780e+02, 6.780e+02,
          6.430e+02],
         [2.350e+02, 2.100e+02, 2.280e+02, ..., 6.440e+02, 6.340e+02,
          6.580e+02],
         [2.510e+02, 2.000e+02, 2.450e+02, ..., 6.680e+02, 6.140e+02,
          6.310e+02]],

        [[1.280e+02, 1.300e+02, 1.340e+02, ..., 1.098e+03, 1.034e+03,
          1.104e+03],
         [1.310e+02, 1.310e+02, 1.340e+02, ..., 1.080e+03, 1.052e+03,
          1.110e+03],
         [1.380e+02, 1.240e+02, 1.150e+02, ..., 1.116e+03, 1.104e+03,
          1.122e+03],
...
         [5.500e+01, 5.400e+01, 5.200e+01, ..., 2.109e+03, 2.149e+03,
         

In [6]:
ds

<xarray.DataArray (time: 14, variable: 13, y: 8020, x: 2335)> Size: 14GB
array([[[[2.380e+02, 2.330e+02, 2.490e+02, ..., 8.310e+02, 8.180e+02,
          8.550e+02],
         [2.540e+02, 2.360e+02, 2.200e+02, ..., 8.140e+02, 8.030e+02,
          8.330e+02],
         [2.540e+02, 2.300e+02, 2.220e+02, ..., 8.310e+02, 8.440e+02,
          8.070e+02],
         ...,
         [2.360e+02, 2.190e+02, 2.160e+02, ..., 6.780e+02, 6.780e+02,
          6.430e+02],
         [2.350e+02, 2.100e+02, 2.280e+02, ..., 6.440e+02, 6.340e+02,
          6.580e+02],
         [2.510e+02, 2.000e+02, 2.450e+02, ..., 6.680e+02, 6.140e+02,
          6.310e+02]],

        [[1.280e+02, 1.300e+02, 1.340e+02, ..., 1.098e+03, 1.034e+03,
          1.104e+03],
         [1.310e+02, 1.310e+02, 1.340e+02, ..., 1.080e+03, 1.052e+03,
          1.110e+03],
         [1.380e+02, 1.240e+02, 1.150e+02, ..., 1.116e+03, 1.104e+03,
          1.122e+03],
...
         [5.500e+01, 5.400e+01, 5.200e+01, ..., 2.109e+03, 2.149e+03,
          2.189e+03],
         [5.500e+01, 5.400e+01, 5.200e+01, ..., 2.109e+03, 2.149e+03,
          2.189e+03],
         [5.500e+01, 5.400e+01, 5.200e+01, ..., 2.109e+03, 2.149e+03,
          2.189e+03]],

        [[6.000e+00, 6.000e+00, 6.000e+00, ..., 4.000e+00, 4.000e+00,
          4.000e+00],
         [6.000e+00, 6.000e+00, 6.000e+00, ..., 4.000e+00, 4.000e+00,
          4.000e+00],
         [6.000e+00, 6.000e+00, 6.000e+00, ..., 4.000e+00, 4.000e+00,
          4.000e+00],
         ...,
         [6.000e+00, 6.000e+00, 6.000e+00, ..., 5.000e+00, 5.000e+00,
          5.000e+00],
         [6.000e+00, 6.000e+00, 6.000e+00, ..., 5.000e+00, 5.000e+00,
          5.000e+00],
         [6.000e+00, 6.000e+00, 6.000e+00, ..., 5.000e+00, 5.000e+00,
          5.000e+00]]]], dtype=float32)
Coordinates:
  * time         (time) datetime64[ns] 112B 2024-03-08 2024-03-21 ... 2024-09-02
  * x            (x) float64 19kB 2.394e+05 2.394e+05 ... 2.628e+05 2.628e+05
  * y            (y) float64 64kB 6.641e+06 6.641e+06 ... 6.56e+06 6.56e+06
    spatial_ref  int64 8B 0
  * variable     (variable) <U3 156B 'B2' 'B3' 'B4' 'B8' ... 'B1' 'B9' 'SCL'
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('B2', 'B3', 'B4', 'B8', 'B5', 'B6', 'B7', 'B8A', 'B11', ...

In [9]:
# 📤 Carpeta de salida
output_folder = os.path.join(data_folder, "merged")
os.makedirs(output_folder, exist_ok=True)


In [ ]:
# Save as NetCDF
ds.to_netcdf(output_folder + "/sentinel2_timeseries.nc")

<ipython-input-10-f09f10e37e63>:2: SerializationWarning: saving variable __xarray_dataarray_variable__ with floating point data as an integer dtype without any _FillValue to use for NaNs
  ds.to_netcdf(output_folder + "/sentinel2_timeseries.nc")
